In [1]:
import os
import pandas
import tqdm
import functools
from sklearn import preprocessing

In [2]:
source = '../resource/kaggle/sample/csv'
table = {
    'article':pandas.read_csv(os.path.join(source, 'articles.csv'), dtype=str),
    'customer':pandas.read_csv(os.path.join(source, 'customers.csv'), dtype=str),
    'transaction':pandas.read_csv(os.path.join(source, 'transactions_train.csv'), dtype=str),
    'submission':pandas.read_csv(os.path.join(source, 'sample_submission.csv'), dtype=str)
}

# Missing

In [3]:
table['article']['detail_desc'].fillna('miss_detail_desc', inplace=True)
table['customer']['FN'].fillna("0.0", inplace=True)
table['customer']['Active'].fillna("0.0", inplace=True)
table['customer']['fashion_news_frequency'].fillna("NONE", inplace=True)
table['customer']['age'].fillna("-1", inplace=True)
table['customer']['postal_code'].fillna('miss_postal_code', inplace=True)

# Numeralization

In [4]:
table['article']["article_label"] = table['article']['article_id']
for key in table['article']:

    if(key=='article_id'): continue

    skip = 10
    engine = preprocessing.LabelEncoder()
    engine.fit(table['article'][key])
    value = engine.transform(table['article'][key])
    table['article'][key] = value + skip
    continue

In [5]:
table['customer']['FN'] = table['customer']['FN'].astype(float)
table['customer']['Active'] = table['customer']['Active'].astype(float)
table['customer']['age'] = table['customer']['age'].astype(float) / 100
for key in table['customer']:

    if(key in ['customer_id', "FN", "Active", 'age']): continue

    skip = 10
    engine = preprocessing.LabelEncoder()
    engine.fit(table['customer'][key])
    value = engine.transform(table['customer'][key])
    table['customer'][key] = value + skip
    continue

In [6]:
table['transaction']['t_dat'] = pandas.to_datetime(table['transaction']['t_dat'])
value = table['transaction']['price'].astype(float).copy()
value = value / value.max()
table['transaction']['price'] = value
for key in table['transaction']:
    
    if(key in ['t_dat', "customer_id", "article_id", 'price']): continue

    skip = 10
    engine = preprocessing.LabelEncoder()
    engine.fit(table['transaction'][key])
    value = engine.transform(table['transaction'][key])
    table['transaction'][key] = value + skip
    continue

# Feature with user base

In [7]:
paste = lambda x: " ".join(x)

In [8]:
sequence = []

In [9]:
table['transaction']['price'] = table['transaction']['price'].astype(str)
table['transaction']['sales_channel_id'] = table['transaction']['sales_channel_id'].astype(str)
sequence += [table['transaction'].groupby(['customer_id', 't_dat'])['price'].apply(paste).reset_index().drop(columns='t_dat')]
sequence += [table['transaction'].groupby(['customer_id', 't_dat'])['sales_channel_id'].apply(paste).reset_index().drop(columns='t_dat')]

In [10]:
for key in tqdm.tqdm(table['article'].keys()):

    if(key=='article_id'): continue

    table['article'][key] = table['article'][key].astype(str)
    selection = table['transaction'][['t_dat', "customer_id", "article_id"]].copy()
    selection = pandas.merge(selection, table['article'][["article_id", key]], on="article_id", how='inner')
    sequence += [selection.groupby(['customer_id', 't_dat'])[key].apply(paste).reset_index().drop(columns='t_dat')]
    continue

100%|██████████| 26/26 [04:05<00:00,  9.46s/it]


In [12]:
merge = lambda x,y: pandas.merge(left=x, right=y, on='customer_id', how='inner')
functools.reduce(merge, sequence[0:5])
# sequence = functools.reduce(merge, sequence)
# sequence.head(4)

,customer_id,price,sales_channel_id,product_code,prod_name,product_type_no
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.11458452722063038 0.0429512893982808 0.05727...,11 11 11,10862 5520 10778,2492 10371 11591,39 31 25
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.11458452722063038 0.0429512893982808 0.05727...,11 11 11,10862 5520 10778,2492 10371 11591,18 18
2,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.11458452722063038 0.0429512893982808 0.05727...,11 11 11,10862 5520 10778,2492 10371 11591,18 31
3,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.11458452722063038 0.0429512893982808 0.05727...,11 11 11,10862 5520 10778,2492 10371 11591,18
4,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.11458452722063038 0.0429512893982808 0.05727...,11 11 11,10862 5520 10778,13709 7107,39 31 25
...,...,...,...,...,...,...
9877980,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,0.06873925501432664 0.03722063037249284 0.0429...,11 11 11 11 11,15253 3028 4770 2065 3027,8703 8681 13935 4210 8752,37 36 36 31 36
9877981,fffec3dbcc87c78300f37f62cfca4274f1ea6ef59dba10...,0.01716332378223496 0.02289398280802292,11 11,3665 12608,5682 9793,61 39
9877982,fffed13f041cc91f74dc6ec4e3267d7bb71bc833d53b7d...,0.14323782234957022,11,13086,13506,36
9877983,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0.10885386819484241,10,14106,2876,101


In [ ]:
sequence

In [ ]:
table['feature'] = pandas.merge(left=table['customer'], right=sequence, on='customer_id', how='outer')

In [ ]:
folder = '../resource/preprocess/csv/'
os.makedirs(folder, exist_ok=True)
table['feature'].dropna().to_csv(os.path.join(folder, "feature(train).csv"), index=False)
table['feature'].fillna("").to_csv(os.path.join(folder, "feature(all).csv"), index=False)